In [1]:
import pickle
import pandas as pd
import numpy as np

In [3]:
from keras.models import Sequential,Input,Model
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import LSTM

Using TensorFlow backend.


In [2]:
with open("../data/sim_data_162_1000_5.pkl","rb") as f:
    sim_data = pickle.load(f)

In [4]:
sim_data.shape

(162000, 5)

In [5]:
sim_data.head()

,pre_mean,pre1,pre2,pre3,y
12,227567.0,10103625.0,397843.0,167289.0,8259355.0
13,329525.0,8259355.0,10103625.0,397843.0,2742105.0
14,162264.0,2742105.0,8259355.0,10103625.0,2915779.0
15,79480.0,2915779.0,2742105.0,8259355.0,758981.0
16,29489.0,758981.0,2915779.0,2742105.0,143696.0


## Data:
The data has 1000 realizations from a mechanistic model, each has 162 data points, each has monthly incidence data from `2003-1~2017-6`.The inputs are previous 3 months, and the mean for previous same months. Output is the next month. <br>
I will use 800 for training and validation, 200 for test. <br>
I will use a batch size of 6 so that each batch contains realization from the same realization. 

## Data Preprocessing

In [22]:
np.argwhere(np.isnan(sim_data["pre_mean"]))

array([[ 77607],
       [113405]])

### reason
Only one previous month, which happens to be 0. Maybe better to take it out in the future. 

In [40]:
sim_data = sim_data.fillna(value=0)

In [41]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
sim_data_scaled = scaler.fit_transform(sim_data)

In [42]:
type(sim_data_scaled)

numpy.ndarray

In [67]:
train_X, train_y = sim_data_scaled[:129600,:-1], sim_data_scaled[:129600,:1]
test_X, test_y = sim_data_scaled[129600:,:-1], sim_data_scaled[129600:,:1]
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(129600, 4) (129600, 1) (32400, 4) (32400, 1)


In [68]:
train_X, test_X = train_X.reshape(train_X.shape[0],1,train_X.shape[1]), test_X.reshape(test_X.shape[0],1,test_X.shape[1])
print(train_X.shape, test_X.shape)

(129600, 1, 4) (32400, 1, 4)


In [78]:
neurons = 4
batch_s = 1
model = Sequential()
model.add(LSTM(neurons,input_shape = (train_X.shape[1], train_X.shape[2])))
model.add(Dense(10))
model.add(Dense(1))
model.compile(loss="mean_squared_error", optimizer = "adam")
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 4)                 144       
_________________________________________________________________
dense_17 (Dense)             (None, 10)                50        
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 11        
Total params: 205
Trainable params: 205
Non-trainable params: 0
_________________________________________________________________


In [79]:
train_X.shape

(129600, 1, 4)

In [80]:
model.fit(train_X,train_y,epochs=20, batch_size=162)

Epoch 1/20
129600/129600 [==============================] - 2s 15us/step - loss: 3.5301e-04
Epoch 2/20
129600/129600 [==============================] - 1s 10us/step - loss: 6.2642e-07
Epoch 3/20
129600/129600 [==============================] - 1s 10us/step - loss: 3.2628e-07
Epoch 4/20
129600/129600 [==============================] - 1s 10us/step - loss: 1.8701e-07
Epoch 5/20
129600/129600 [==============================] - 1s 10us/step - loss: 1.6605e-07
Epoch 6/20
129600/129600 [==============================] - 1s 10us/step - loss: 1.8085e-07
Epoch 7/20
129600/129600 [==============================] - 1s 10us/step - loss: 2.0243e-07
Epoch 8/20
129600/129600 [==============================] - 1s 10us/step - loss: 2.0369e-07
Epoch 9/20
129600/129600 [==============================] - 1s 10us/step - loss: 2.1885e-07
Epoch 10/20
129600/129600 [==============================] - 1s 10us/step - loss: 1.8930e-07
Epoch 11/20
129600/129600 [==============================] - 1s 10us/step - los

In [54]:
train_X[:20,:]

array([[[0.01162288],
        [0.19476609],
        [0.00766916],
        [0.00322481]],

       [[0.01683034],
        [0.15921437],
        [0.19476609],
        [0.00766916]],

       [[0.00828756],
        [0.05285915],
        [0.15921437],
        [0.19476609]],

       [[0.0040594 ],
        [0.05620704],
        [0.05285915],
        [0.15921437]],

       [[0.00150614],
        [0.01463076],
        [0.05620704],
        [0.05285915]],

       [[0.00270843],
        [0.00277001],
        [0.01463076],
        [0.05620704]],

       [[0.00212725],
        [0.00468759],
        [0.00277001],
        [0.01463076]],

       [[0.00838705],
        [0.00311874],
        [0.00468759],
        [0.00277001]],

       [[0.01092127],
        [0.00183198],
        [0.00311874],
        [0.00468759]],

       [[0.00854421],
        [0.00138751],
        [0.00183198],
        [0.00311874]],

       [[0.02031965],
        [0.0010385 ],
        [0.00138751],
        [0.00183198]],

       [[0